In [5]:
import MySQLdb
import torch
import numpy as np
import matplotlib.pyplot as plt
import stock
import re
from IPython.display import display, Markdown

In [6]:
#1. 忽略掉不想要的类型
#2. 忽略掉在一年内有连续跌停的
#3. 忽略掉ST
#符合条件的filter返回True
keywords = [r".*证券.*",r".*银行.*",r".*芯片.*",r".*半导体.*",r".*集成电路.*",r".*互联网.*",
            r".*制药.*",r".*固废.*",r".*通信.*"]
def descFilter(c):
    for pat in keywords:
        if re.match(pat,c[3]) is not None:
            return True
    return False
def IgnoreCompany(filter=descFilter):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    cursor = db.cursor()
    try:
        cursor.execute("""update company set `ignore`=1""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
        
    db.query("""select * from descript where name not like '%ST%' and name not like '%B%'""")
    r = db.store_result()
    companys = r.fetch_row(r.num_rows())
    for c in companys:
        if filter(c):
            try:
                cursor.execute("""update company set `ignore`=0 where code='%s'"""%(c[2]))
                db.commit()
            except Exception as e:
                print(e)
                db.rollback()
    try:
        cursor.execute("""select count(*) from company where `ignore`=0""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
    result = cursor.fetchone()
    db.close()
    print(result)
    
def loadCompany():
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    db.query("""select * from company where `ignore`=0""")
    r = db.store_result()
    companys = r.fetch_row(r.num_rows())
    db.close()
    return companys

# 使用boll 极限宽带小于0.1来选择股票

In [9]:
def bollWidth(k):
    bo20 = stock.bollLine(k,20)
    lw = (bo20[:,2]-bo20[:,0])/bo20[:,1]
    lwmax = 0
    lwmin = 10
    #boll极限宽小于0.1
    for i in range(len(lw)-10,len(lw)-1):
        if lw[i] > lwmax:
            lwmax = lw[i]
        if lw[i] < lwmin:
            lwmin = lw[i]
    #返回最小的极限宽带和最大的极限宽带
    return lwmin,lwmax,lw[-1],k[-1,4]-k[-2,4],k[-1,4]-bo20[-1,1]

def storeSelectCompany(blist):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
    cursor = db.cursor()
    try:
        cursor.execute("""delete from company_select2 where source=1""")
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()
    for c in blist:
        try:
            cursor.execute("""insert ignore into company_select2 values ('%s','%s',1)"""%(c[0],c[1]))
            db.commit()
        except Exception as e:
            print(e)
            db.rollback()        
    db.close()
def ouputSelectCompany(str,blist):
    display(Markdown("""## %s %d"""%(str,len(blist))))
    for b in blist:
        display(Markdown("""[%s %s](https://xueqiu.com/S/%s)"""%(b[0],b[1],b[0])))
        
def selectList():
    companys = loadCompany()
    glist = []
    nlist = []
    blist = []
    llist = []
    for com in companys:
        c,kline,kdate = stock.loadKline(com[1])
        lwmin,lwmax,lwlast,delta,cm20 = bollWidth(kline)
        if lwmax < 0.1 and lwmax/lwmin < 1.5:
            if lwlast > lwmax and lwlast/lwmax>1.2:
                if delta>0:
                    blist.append([com[1],com[2]])
                else:
                    llist.append([com[1],com[2]])
            else:
                if cm20 > 0:
                    glist.append([com[1],com[2]])
                else:
                    nlist.append([com[1],com[2]])
    storeSelectCompany(blist+llist+glist+nlist)
    ouputSelectCompany("打开向上",blist)
    ouputSelectCompany("打开向下",llist)
    ouputSelectCompany("通道上",glist)
    ouputSelectCompany("通道下",nlist)

In [10]:
selectList()

## 打开向上 2

[SH603222 济民制药](https://xueqiu.com/S/SH603222)

[SZ002230 科大讯飞](https://xueqiu.com/S/SZ002230)

## 打开向下 0

## 通道上 48

...

[SH600575 淮河能源](https://xueqiu.com/S/SH600575)

[SZ000516 国际医学](https://xueqiu.com/S/SZ000516)

[SZ002349 精华制药](https://xueqiu.com/S/SZ002349)

[SZ002424 贵州百灵](https://xueqiu.com/S/SZ002424)

[SH600388 龙净环保](https://xueqiu.com/S/SH600388)